In [3]:
import os
import json
import pandas as pd
from typing import Dict, List, Optional, Union, cast
import requests
# import prepare
from env import token, username

In [48]:
# import requests
# from env import token, username

# GitHub API endpoint
url = "https://api.github.com/search/repositories"

# Parameters for the API request
params = {
    "q": "stars:>1000",
    "sort": "stars",
    "order": "desc",
    "per_page": 100,
}

# Authentication headers
headers = {"Authorization": f"token {token}", "User-Agent": username}

# Send a GET request to the GitHub API
response = requests.get(url, params=params, headers=headers)

# Check if the response is successful
if response.status_code == 200:
    # Get the response data
    response_data = response.json()

    # Extract the repository names
    repos = [repo["full_name"] for repo in response_data["items"]]

    # Display the repository names
#     print(repos)
else:
    print("Failed to retrieve repository names from the GitHub API")
    
# Convert repos list to DataFrame
df_repos = pd.DataFrame({'Repository': repos})

# Display the DataFrame
df_repos

,Repository
0,freeCodeCamp/freeCodeCamp
1,EbookFoundation/free-programming-books
2,996icu/996.ICU
3,jwasham/coding-interview-university
4,sindresorhus/awesome
...,...
95,neovim/neovim
96,thedaviddias/Front-End-Checklist
97,Hack-with-Github/Awesome-Hacking
98,florinpop17/app-ideas


In [41]:
# Scrape README files and store their content in a dictionary
readme_data = {'Repository': [], 'Readme': []}

for repo in repos:
    # Get the URL of the README file
    url = f"https://api.github.com/repos/{repo}/readme"

    # Send a GET request to the URL with authentication
    headers = {"Authorization": f"token {token}", "User-Agent": username}
    response = requests.get(url, headers=headers)

    # Check if the response is successful
    if response.status_code == 200:
        # Get the content from the response
        response_data = response.json()
        readme_content = response_data['content']

        # Decode the base64 encoded content
        import base64
        readme_text = base64.b64decode(readme_content).decode('utf-8')

        # Store the repository name and the corresponding README content
        readme_data['Repository'].append(repo)
        readme_data['Readme'].append(readme_text)
    else:
        print(f"Failed to retrieve README for repository: {repo}")


In [42]:
# checking the dict
readme_data

{'Repository': ['EbookFoundation/free-programming-books',
  'twbs/bootstrap',
  'ossu/computer-science',
  'jlevy/the-art-of-command-line',
  'ytdl-org/youtube-dl',
  'FortAwesome/Font-Awesome',
  'Hack-with-Github/Awesome-Hacking',
  'ansible/ansible',
  'chrislgarry/Apollo-11',
  'Anduin2017/HowToCook',
  'junegunn/fzf',
  'apache/superset',
  'microsoft/ML-For-Beginners',
  'obsproject/obs-studio',
  'binhnguyennus/awesome-scalability',
  'minimaxir/big-list-of-naughty-strings',
  'pandas-dev/pandas',
  'hexojs/hexo',
  'TanStack/query',
  'preactjs/preact',
  'photonstorm/phaser',
  'airbnb/lottie-android',
  'SheetJS/sheetjs',
  'koalaman/shellcheck',
  'dbeaver/dbeaver',
  'Light-City/CPlusPlusThings',
  'markedjs/marked',
  'xkcoding/spring-boot-demo',
  'square/leakcanary',
  'codepath/android_guides',
  'pcottle/learnGitBranching',
  'ggerganov/llama.cpp',
  'angular/angular-cli',
  'codemirror/codemirror5',
  'digitalocean/nginxconfig.io',
  'Automattic/mongoose',
  'labstack

In [11]:
github_token = token
github_username = username

In [14]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

In [17]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_contents = requests.get(get_readme_download_url(contents)).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in repos]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [22]:
if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [23]:
data

[{'repo': 'EbookFoundation/free-programming-books',
  'language': None,
  'readme_contents': '# List of Free Learning Resources In Many Languages\n\n<div align="center" markdown="1">\n\n[![Awesome](https://cdn.rawgit.com/sindresorhus/awesome/d7305f38d29fed78fa85652e3a63e154dd8e8829/media/badge.svg)](https://github.com/sindresorhus/awesome)&#160;\n[![License: CC BY 4.0](https://img.shields.io/badge/License-CC%20BY%204.0-lightgrey.svg)](https://creativecommons.org/licenses/by/4.0/)&#160;\n[![Hacktoberfest 2022 stats](https://img.shields.io/github/hacktoberfest/2022/EbookFoundation/free-programming-books?label=Hacktoberfest+2022)](https://github.com/EbookFoundation/free-programming-books/pulls?q=is%3Apr+is%3Amerged+created%3A2022-10-01..2022-10-31)\n\n</div>\n\nSearch the list at [https://ebookfoundation.github.io/free-programming-books-search/](https://ebookfoundation.github.io/free-programming-books-search/) [![https://ebookfoundation.github.io/free-programming-books-search/](https://im

In [40]:
for i in range(len(data)):
    print(f'Dionne REPOS: {REPOS[i]}, Repo Name: {data[i]["repo"]}, Language: {data[i]["language"]}')

Dionne REPOS: EbookFoundation/free-programming-books, Repo Name: EbookFoundation/free-programming-books, Language: None
Dionne REPOS: twbs/bootstrap, Repo Name: twbs/bootstrap, Language: JavaScript
Dionne REPOS: ossu/computer-science, Repo Name: ossu/computer-science, Language: None
Dionne REPOS: jlevy/the-art-of-command-line, Repo Name: jlevy/the-art-of-command-line, Language: None
Dionne REPOS: ytdl-org/youtube-dl, Repo Name: ytdl-org/youtube-dl, Language: Python
Dionne REPOS: FortAwesome/Font-Awesome, Repo Name: FortAwesome/Font-Awesome, Language: JavaScript
Dionne REPOS: Hack-with-Github/Awesome-Hacking, Repo Name: Hack-with-Github/Awesome-Hacking, Language: None
Dionne REPOS: ansible/ansible, Repo Name: ansible/ansible, Language: Python
Dionne REPOS: chrislgarry/Apollo-11, Repo Name: chrislgarry/Apollo-11, Language: Assembly
Dionne REPOS: Anduin2017/HowToCook, Repo Name: Anduin2017/HowToCook, Language: HTML
Dionne REPOS: junegunn/fzf, Repo Name: junegunn/fzf, Language: Go
Dionne R

In [43]:
def get_repo_names():
    '''
    Gets a list of repo names from github
    Modules: 
        import requests
        from env import token, username
    '''

    # GitHub API endpoint
    url = "https://api.github.com/search/repositories"

    # Parameters for the API request
    params = {
        "q": "stars:>0",
        "sort": "stars",
        "order": "desc",
        "per_page": 100,
    }

    # Authentication headers needed for api access
    headers = {"Authorization": f"token {token}", "User-Agent": username}

    # Send a GET request to the GitHub API
    response = requests.get(url, params=params, headers=headers)

    # Check if the response is successful
    if response.status_code == 200:
        
        # Get the response data
        response_data = response.json()

        # Extract the repository names
        repos = [repo["full_name"] for repo in response_data["items"]]
        
    # if reposne unsuccessful    
    else:
        
        # notify the user
        print("Failed to retrieve repository names from the GitHub API")
    
    # exit and returns the lsit of repos to use
    return repos

In [44]:
# testing
repos = get_repo_names()

#### Takeaways:
* Something about the way the repo list is grabbed causes many shifts depending on when you pull that data. This will need to be invesitgated further